In [ ]:
import pandas as pd
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim 
import gensim.downloader as api
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv(.....)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [ ]:
w2v_model = gensim.models.Word2Vec(X_train, window = 5, min_count = 2)

In [ ]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(new_features_df['alignment'])
encoded_Y = encoder.transform(new_features_df['alignment'])
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
rnn_X_train, rnn_X_test, rnn_y_train, rnn_y_test = train_test_split(df["full_text"], dummy_y)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(rnn_X_train)

X_train_seq = tokenizer.texts_to_sequences(rnn_X_train)
X_test_seq = tokenizer.texts_to_sequences(rnn_X_test)

In [ ]:
import keras.backend as K
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential

def recall(y_true, y_pred):
    true_pos = K.sum(K.round(K.clip(y_true*y_pred, 0, 1)))
    poss_pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_pos/(poss_pos + K.epsilon())
    return recall
    
def precision(y_true, y_pred):
    true_pos = K.sum(K.round(K.clip(y_true*y_pred, 0, 1)))
    pred_pos = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_pos/(pred_pos + K.epsilon())
    return precision

In [ ]:
model = Sequential()

model.add(Embedding(len(tokenizer.index_word) + 1, 32))
model.add(LSTM(32, dropout = .2, recurrent_dropout = 0))
model.add(Dense(32, activation = "relu"))
model.add(Dense(3, activation = "softmax"))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', precision_m, recall_m])

In [ ]:
history = model.fit(X_train_seq, rnn_y_train, 
                    batch_size=32, epochs=10,
                    validation_data=(X_test_seq, rnn_y_test))